# Trendline for model 

In [ ]:
from core.gsheet_utils import *
from core.sql_utils import *
from core.sql_utils import get_connection, get_sqlalchemy_engine
import pandas as pd
from core.gsheet_utils import load_excel_data
from sqlalchemy import text
from results.trendline.main import generate_trendline_functions
from core.plt_utils import show_trendline
from core.sql_utils import *
from activation.config.mappings import mapping_vehicle_type
from results.trendline.trendline_utils import filtrer_trendlines

### Load data

In [ ]:

engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT vm.model_name, vm.id, vm.type, m.make_name, b.capacity FROM vehicle_model vm
                                  join make m on m.id=vm.make_id
                                  join battery b on b.id=vm.battery_id;"""), con)


In [ ]:

with engine.connect() as connection:
    prod_df = pd.read_sql(text("""SELECT v.vin, vm.model_name, vm.type, vd.odometer, vd.soh, vm.version  from vehicle_data vd
join vehicle v
on vd.vehicle_id=v.id
join vehicle_model vm
on v.vehicle_model_id=vm.id;"""), con)


In [ ]:
df = load_excel_data("Courbes de tendance", "Courbes OS")
df_sheet = pd.DataFrame(columns=['make_name', 'model', 'type', 'year', 'odometer', 'soh', 'vin', 'battery_capacity'], data=df[1:,:8])
df_sheet['soh'] = df_sheet['soh'].apply(lambda x:  x.replace('%', '').strip()).astype(float) / 100
df_sheet['odometer'] = df_sheet['odometer'].apply(lambda x: float(x.replace(',', '')))

In [ ]:
df_sheet['model_id'] = df_sheet.apply(lambda row: mapping_vehicle_type(row['type'], row['make_name'], 
                                                                       row['model'], dbeaver_df), axis=1)
df_sheet['model_id'] = df_sheet['model_id'].astype(str)

## Check conditioning

In [ ]:
model_ok = {}
for modele, group in df_sheet.groupby(["model","type", 'model_id']):
    print(modele)
    if filtrer_trendlines(group, km_lower=50_000, km_upper=50_000, vin_total=20, nbr_under=10, nbr_upper=10):
        model_ok[(modele[0], modele[1])] =  generate_trendline_functions(df_sheet[df_sheet['model_id']==modele[2]], 'odometer', "soh")


In [ ]:
for model in model_ok:
    fig = show_trendline(df_sheet[(df_sheet['model']==model[0]) & (df_sheet['type']==model[1])], model_ok[model][0], model_ok[model][1], model_ok[model][2], model ,'odometer', "soh")
    fig.show()